<a href="https://colab.research.google.com/github/vifirsanova/100-days-of-code/blob/main/day18/Topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Автор: [Виктория Фирсанова](https://github.com/vifirsanova)

# Импорт библиотек

In [ ]:
# цветной вывод
!pip install colorama

In [ ]:
# последняя версия spacy
!pip install -U spacy

In [ ]:
# spacy для русского языка
!spacy download ru_core_news_sm

In [ ]:
pip install pyLDAvis

In [ ]:
import pandas as pd
import json
from colorama import Fore, Style

import re

import nltk
from nltk.corpus import stopwords

import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel

#import spacy
#import ru_core_news_sm

from pprint import pprint

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

# Загрузка данных

Для обучения используем собственный [корпус новостей газеты Московский комсомолец за 2020-2021 года.](https://drive.google.com/file/d/1ebxe77hTEWvmUggCMnRGdrVDwhuxC9Mr/view)

In [ ]:
# преобразование данных корпуса в таблицу csv
with open('corpus.json') as json_file:
    data = json.load(json_file)
with open('corpus_data.json', 'w') as json_file:
    json.dump(data['data'], json_file)
df = pd.read_json('corpus_data.json')
print(f"{Fore.RED}Список тем в корпусе:{Style.RESET_ALL} {df.rubric.unique()}")
df.head()

Список тем в корпусе: ['Общество' 'Происшествия' 'Политика' 'Москва' 'Экономика' 'Наука'
 'В мире' 'Спорт' 'Московская область' 'Культура' 'Армия' 'Авто' 'Космос'
 'Светская жизнь' 'Технологии' 'Рунет' 'Кошелек' 'Футбол' 'Катастрофы'
 'Образование' 'Письма президенту' 'Здоровье' 'Московская премьера'
 'Свежий МК' 'Турклуб' 'Криминал']


,link,title,date,time,rubric,text
0,https://www.mk.ru/social/2020/03/31/minobrnauk...,Минобрнауки решило перенести вступительные экз...,31.03.2020,09:11,Общество,\nВ Минобрнауки РФ приняли решение перенести д...
1,https://www.mk.ru/social/2020/08/17/razveyan-m...,Развеян миф о риске заразиться COVID-19 через ...,17.08.2020,09:33,Общество,\nВсемирная организация здравоохранения (ВОЗ) ...
2,https://www.mk.ru/incident/2020/05/05/v-gaage-...,В Гааге задержали десятки протестующих против ...,05.05.2020,17:12,Происшествия,\nПолиция Нидерландов пресекла акцию протеста ...
3,https://www.mk.ru/social/2021/03/23/v-kremle-p...,В Кремле прокомментировали возможную отставку ...,23.03.2021,13:05,Общество,\nПресс-секретарь российского президента Дмитр...
4,https://www.mk.ru/social/2021/03/01/moskva-i-p...,Москва и Петербург не попали в топ-50 городов ...,01.03.2021,12:17,Общество,\nСоставлен рейтинг 100 лучших городов мира по...


# Токенизация

In [ ]:
# сохраняем тексты в переменную (list)
data = df.text.values.tolist()
# удаляем латинские буквы, т.к. корпус содержит русскоязычные новости, а вставки на латинице могут содержать ссылки, элементы сайта (например, рекламные вставки) и другие побочные элементы
data = [re.sub('[a-zA-Z]', '', sent) for sent in data]

# создаем функцию для токенизации
def tokenize(data):
    for text in data:
        # для токенизации используем simple_preprocess из gensim
        yield(gensim.utils.simple_preprocess(str(text).encode('utf-8'), deacc=True))  # deacc удаляет знаки препинания

data_tokenized = list(tokenize(data))
print(f"{Fore.RED}Образец токенизированного текста{Style.RESET_ALL}\n", data_tokenized[0])

Образец токенизированного текста
 ['минобрнауки', 'рф', 'приняли', 'решение', 'перенести', 'даты', 'вступительных', 'экзаменов', 'вузы', 'из', 'за', 'коронавируса', 'заявил', 'глава', 'ведомства', 'валерии', 'фальков', 'интервью', 'россия', 'он', 'отметил', 'что', 'этом', 'году', 'число', 'бюджетных', 'мест', 'было', 'увеличено', 'на', 'тысячи', 'сеичас', 'начался', 'экономическии', 'спад', 'но', 'менять', 'стратегию', 'министерстве', 'не', 'намерены', 'решение', 'об', 'увеличении', 'бюджетных', 'мест', 'вузах', 'связано', 'ростом', 'числа', 'выпускников', 'школ', 'будет', 'увеличено', 'число', 'мест', 'на', 'педагогическом', 'медицинском', 'направлениях', 'большои', 'запрос', 'на', 'бюджетные', 'места', 'регионов', 'подчеркнул', 'фальков', 'ранее', 'власти', 'перенесли', 'даты', 'егэ', 'огэ', 'читаите', 'также', 'скворцова', 'похвалила', 'россиискии', 'препарат', 'от', 'коронавируса']


# Стоп-слова

In [ ]:
# Загрузка стоп-слов для русского языка из NLTK
nltk.download('stopwords')
stop_words = stopwords.words('russian')
stop_words[:10]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

# Биграммная модель

In [ ]:
# Используем Phrases из gensim
# на вход модель принимает токенизированные данные
# чем выше min_count и threshold, тем меньше биграм будет устанавливать модель
bigram = gensim.models.Phrases(data_tokenized, min_count=5, threshold=100)
model = gensim.models.phrases.Phraser(bigram)

print(f"{Fore.RED}Образец работы биграммной модели{Style.RESET_ALL}\n", model[model[data_tokenized[0]]])

Образец работы биграммной модели
 ['минобрнауки', 'рф', 'приняли_решение', 'перенести', 'даты', 'вступительных', 'экзаменов', 'вузы', 'из', 'за', 'коронавируса', 'заявил', 'глава', 'ведомства', 'валерии_фальков', 'интервью', 'россия', 'он', 'отметил', 'что', 'этом', 'году', 'число', 'бюджетных_мест', 'было', 'увеличено', 'на', 'тысячи', 'сеичас', 'начался', 'экономическии_спад', 'но', 'менять', 'стратегию', 'министерстве', 'не', 'намерены', 'решение', 'об_увеличении', 'бюджетных_мест', 'вузах', 'связано', 'ростом_числа', 'выпускников_школ', 'будет', 'увеличено', 'число', 'мест', 'на', 'педагогическом', 'медицинском', 'направлениях', 'большои', 'запрос', 'на', 'бюджетные', 'места', 'регионов', 'подчеркнул', 'фальков', 'ранее', 'власти', 'перенесли', 'даты', 'егэ', 'огэ', 'читаите', 'также', 'скворцова', 'похвалила', 'россиискии', 'препарат', 'от', 'коронавируса']


# Преобработка

In [ ]:
def remove_stopwords(data):
    """
    Удаление стоп-слов
    :data: токенизированные тексты (list)
    :return: токенизированные тексты, очищенные от стоп-слов (list)
    """
    return [[word for word in text if word not in stop_words] for text in data]


def bigrams(data):
    """
    Построение биграмм
    :data: токенизированные тексты (list)
    :return: результат работы биграммной модели (list)
    """
    return [model[text] for text in data]


# Лемматизация с помощью Spacey
# здесь мы ее пропустили из-за объема корпуса
##def lemmatization(data):
##    nlp = spacy.load('ru_core_news_sm')
##    data_lemmatized = []
##    for sent in data:
##        text = nlp(" ".join(sent)) 
##        data_lemmatized.append([token.lemma_ for token in text])
##    return data_lemmatized


data_clean = remove_stopwords(data_tokenized)
data_bigrams = bigrams(data_clean)
##data_lemmatized = lemmatization(data_bigrams)


print(f"{Fore.RED}Образец очищенных от стоп-слов данных:{Style.RESET_ALL}\n", data_clean[0])
print(f"{Fore.RED}Образец обработанных данных c биграммами:{Style.RESET_ALL}\n", data_bigrams[0])
##print(f"{Fore.RED}Образец лемматизированных данных:{Style.RESET_ALL}\n", data_lemmatized[0])

Образец очищенных от стоп-слов данных:
 ['минобрнауки', 'рф', 'приняли', 'решение', 'перенести', 'даты', 'вступительных', 'экзаменов', 'вузы', 'коронавируса', 'заявил', 'глава', 'ведомства', 'валерии', 'фальков', 'интервью', 'россия', 'отметил', 'году', 'число', 'бюджетных', 'мест', 'увеличено', 'тысячи', 'сеичас', 'начался', 'экономическии', 'спад', 'менять', 'стратегию', 'министерстве', 'намерены', 'решение', 'увеличении', 'бюджетных', 'мест', 'вузах', 'связано', 'ростом', 'числа', 'выпускников', 'школ', 'увеличено', 'число', 'мест', 'педагогическом', 'медицинском', 'направлениях', 'большои', 'запрос', 'бюджетные', 'места', 'регионов', 'подчеркнул', 'фальков', 'ранее', 'власти', 'перенесли', 'даты', 'егэ', 'огэ', 'читаите', 'также', 'скворцова', 'похвалила', 'россиискии', 'препарат', 'коронавируса']
Образец обработанных данных c биграммами:
 ['минобрнауки', 'рф', 'приняли_решение', 'перенести', 'даты', 'вступительных', 'экзаменов', 'вузы', 'коронавируса', 'заявил', 'глава', 'ведомств

# Обучающий словарь и Term Frequency

In [ ]:
# создание словаря с помощью gensim.corpora
id2word = corpora.Dictionary(data_bigrams)

# TF
corpus = [id2word.doc2bow(text) for text in data_bigrams]

print(f"{Fore.RED}Образец элементов словаря:{Style.RESET_ALL}\n", id2word[0], id2word[1], id2word[2])
print(f"{Fore.RED}Образец обучающего корпуса:{Style.RESET_ALL}\n", corpus[0])
print(f"{Fore.RED}Term Frequency:{Style.RESET_ALL}\n", [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]])

Образец элементов словаря:
 большои бюджетные бюджетных_мест
Образец обучающего корпуса:
 [(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 2), (51, 1), (52, 2), (53, 1), (54, 1), (55, 1)]
Term Frequency:
 [[('большои', 1), ('бюджетные', 1), ('бюджетных_мест', 2), ('валерии_фальков', 1), ('ведомства', 1), ('власти', 1), ('вступительных', 1), ('вузах', 1), ('вузы', 1), ('выпускников_школ', 1), ('глава', 1), ('году', 1), ('даты', 2), ('егэ', 1), ('запрос', 1), ('заявил', 1), ('интервью', 1), ('коронавируса', 2), ('медицинском', 1), ('менять', 1), ('мест', 1), ('места', 1), ('министерстве', 1), ('мин

# Модель LDA

In [ ]:
# обучение модели LDA с gensim
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# Результат

In [ ]:
pprint(lda_model.print_topics())

[(0,
  '0.046*"человек" + 0.025*"сообщает" + 0.020*"коронавирусом" + '
  '0.018*"сообщили" + 0.014*"около" + 0.013*"данным" + 0.012*"человека" + '
  '0.011*"тысяч" + 0.010*"данные" + 0.010*"отмечается"'),
 (1,
  '0.045*"время" + 0.018*"сообщила" + 0.017*"москвы" + 0.016*"москве" + '
  '0.013*"трамп" + 0.013*"столице" + 0.011*"теме" + 0.011*"рассказала" + '
  '0.011*"дома" + 0.010*"городе"'),
 (2,
  '0.045*"это" + 0.037*"также" + 0.027*"года" + 0.015*"будут" + 0.011*"однако" '
  '+ 0.009*"кроме" + 0.008*"которая" + 0.007*"компании" + 0.006*"сми" + '
  '0.006*"свои"'),
 (3,
  '0.028*"лет" + 0.014*"тыс" + 0.013*"затем" + 0.010*"сентября" + '
  '0.009*"ребенка" + 0.008*"нового" + 0.007*"суда" + 0.006*"сразу" + '
  '0.006*"получил" + 0.005*"одном"'),
 (4,
  '0.033*"коронавируса" + 0.022*"коронавирус" + 0.014*"сутки" + '
  '0.013*"уточняется" + 0.013*"стало_известно" + 0.012*"ссылкои" + '
  '0.011*"настоящее_время" + 0.010*"находятся" + 0.009*"декабря" + '
  '0.009*"москве"'),
 (5,
  '0.031*

In [ ]:
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
2     -0.224949  0.062004       1        1  9.991691
13    -0.246691  0.104478       2        1  8.689127
16    -0.231537 -0.149315       3        1  7.864947
0     -0.036552  0.288582       4        1  6.261716
17    -0.143420 -0.196483       5        1  6.016813
5      0.017482  0.025320       6        1  5.700159
9      0.016347  0.136818       7        1  4.967184
12     0.055391 -0.039567       8        1  4.435719
1      0.063305  0.031309       9        1  4.407698
15     0.081005 -0.032595      10        1  4.147368
6      0.047103 -0.082278      11        1  4.119358
10     0.031313 -0.062873      12        1  3.960228
7      0.071910 -0.031727      13        1  3.916965
11     0.050559 -0.027066      14        1  3.779091
3      0.086215  0.019334      15        1  3.739536
4      0.044719 -0.019987      16        1  3.694166
19     0.070287 -0.048126      17        1  3.677046
8      0.067193  0.097040      18        1  3.650839
14     0.088734 -0.045174      19        1  3.570976
18     0.091588 -0.029692      20        1  3.409374, topic_info=                 Term          Freq         Total Category  logprob  loglift
47              также  19010.000000  19010.000000  Default  30.0000  30.0000
622            россии  13652.000000  13652.000000  Default  29.0000  29.0000
178               это  13429.000000  13429.000000  Default  28.0000  28.0000
115            словам   8766.000000   8766.000000  Default  27.0000  27.0000
503               сша   7915.000000   7915.000000  Default  26.0000  26.0000
...               ...           ...           ...      ...      ...      ...
3185         кампании    268.318157    269.204489  Topic20  -5.7756   3.3753
9066   первую_очередь    267.709453    268.595786  Topic20  -5.7779   3.3753
2517           проити    263.153378    264.039711  Topic20  -5.7950   3.3753
5135           заводе    258.416251    259.302584  Topic20  -5.8132   3.3752
12678    прошлом_году    257.726476    258.612808  Topic20  -5.8159   3.3752

[750 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
2233      18  0.997441       аварии
572       11  0.998942      августа
8125      20  0.998785  автомобилеи
13561      7  0.998346   автомобиль
2756      16  0.998888    агентства
...      ...       ...          ...
1097      11  0.997212     является
1097      13  0.002322     является
2743       1  0.999724        якобы
2666      15  0.998371       январе
2022      14  0.999261       января

[823 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 14, 17, 1, 18, 6, 10, 13, 2, 16, 7, 11, 8, 12, 4, 5, 20, 9, 15, 19])

# Перплексия, Coherence Score

In [ ]:
# Перплексия
perplexity = lda_model.log_perplexity(corpus)
print('Перплексия: ', perplexity)

# Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_bigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

Перплексия:  -11.5288572920937

Coherence Score:  0.3045778313558452
